### Model testing
Trying to identify how models work in different configurations

In [14]:
from sklearn.preprocessing import LabelEncoder
from src.myscripts import prepare_data
import pandas as pd
import contextlib
import sys
from sklearn.metrics import accuracy_score
from src.myscripts.model import Conv1DClassifier
import os
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import config
from src.myscripts.train import ModelTrainer

data = pd.read_csv(f"{config.DATAFRAMES_DIR_PATH}/sound_data.csv")
splits_path = config.DATA_SPLIT_SAVE_DIR_PATH
path=config.DATA_SPLIT_SAVE_DIR_PATH
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_train, _, _, _, _, _ = prepare_data.prepare_data_for_model(data, -1, LabelEncoder, [0.7, 0.2, 0.1], save_to_npy=False)
input_shape = x_train.shape[1:]


@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as fnull:
        old_stdout = sys.stdout
        try:
            sys.stdout = fnull
            yield
        finally:
            sys.stdout = old_stdout



def evaluate_avarage_model_accuracy(model, device, num_epochs:int, early_stopping_rounds:int, num_of_samples:int):
    model_trainer = ModelTrainer(model, device)
    
    #Przekonwertowanie jezeli etykiety bylyby w One-hot
  
    acc_sum=0
    

    for i in range(num_of_samples):
        x_train, y_train, x_val, y_val, x_test, y_test = prepare_data.prepare_data_for_model(data, -1, LabelEncoder, [0.7, 0.2, 0.1], random_state=np.random.randint(0,10000), save_to_npy=False)
        x_train = x_train.astype(np.float32)
        x_val = x_val.astype(np.float32)
        y_train = y_train.astype("long")
        y_val = y_val.astype("long")
        x_test = x_test.astype(np.float32)
        y_test = y_test.astype("long")
        x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
        y_val_tensor = torch.tensor(y_val, dtype=torch.long)
        train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
        val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
        x_test_tensor = torch.tensor(x_test).to(device)
        if len(y_test.shape) > 1 and y_test.shape[1] > 1:
            true_labels = np.argmax(y_test, axis=1)
        else:
            true_labels = y_test
    
        with suppress_stdout():
            model_trainer.train_model(train_dataset, val_dataset, epochs=num_epochs,early_stopping_rounds=early_stopping_rounds)
        sample_model=model_trainer.get_trained_model()
        sample_model.eval()
        with torch.no_grad():
            outputs = sample_model(x_test_tensor)
            _, predictions = torch.max(outputs, 1)
        
        predictions = predictions.cpu().numpy()
        accuracy = accuracy_score(true_labels, predictions)
        acc_sum+=accuracy
        print(f"\rSamples done: {i+1}/{num_of_samples}\033[K", end="")
    print("\n--------------")
    print(f"Avarage accuracy of model in {num_of_samples} samples is: {acc_sum/num_of_samples}")
    

### ZROBIC TUTAJ TAK ZEBY MOZNA BYLO ROZNE KONFIGURACJE TYCH SKLADANYCH DATAFRAMOW np. ze polaczenie mel i chroma to zeby mozna bylo tutaj napisac zeby uzyc danego polaczenia i Ci stworzy dataframe.

In [17]:
model = Conv1DClassifier(num_classes=43, input_shape=input_shape)
model_trainer = ModelTrainer(model,device)
evaluate_avarage_model_accuracy(model, device, early_stopping_rounds=3, num_epochs=15, num_of_samples=10)

Samples done: 10/10
--------------
Avarage accuracy of model in 10 samples is: 0.9965753424657533
